In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
raw = pd.read_csv('./DATA/데이터2303_2310.csv', encoding='euc-kr', converters={'코드':str})
raw.shape

(13773, 18)

In [2]:
data = raw[(raw['2308차월']>7)&(raw['2308차월']<38)]
data = data[(data.해촉사유.isna() )| ((data.해촉사유=='일반')&(data.해촉일자>20230801))]
data = data[['코드','해촉일자','해촉사유']]
data = data.set_index('코드')
data.head(1)

,해촉일자,해촉사유
코드,,
0001870469,NaN,NaN


# 등급별 이탈률

In [3]:
score = pd.read_excel('../result2차/스코어_2308.xlsx', encoding='euc-kr', converters={'코드':str})
score = score.set_index('코드')
score.head(1)

,score1,pred,그룹,환산,활동1,활동2,case,2308차월
코드,,,,,,,,
0001920598,752,0.006357,1,True,True,True,1,7


In [4]:
data.shape

(5638, 2)

In [5]:
df = data.join(score)
df = df[~df.score1.isna()]
# df.head(1)
df.shape

(5432, 10)

In [25]:
bins = [0]
num_cat = 8
for i in range(num_cat-1):
    bins += [round(df.score1.quantile((i+1)/num_cat))]
bins += [max(df.score1)]
bins

[0, 323.0, 416.0, 509.0, 564.0, 602.0, 632.0, 663.0, 747.0]

In [26]:
labels = list(range(1, len(bins)))
df['grade'] = pd.cut(df['score1'], bins=bins, labels=labels, include_lowest=True)
df['y'] = list(map(lambda x: 1 if x=='일반' else 0, df.해촉사유))
df.head(1)

,해촉일자,해촉사유,y,score1,pred,그룹,환산,활동1,활동2,case,2309차월,grade,FC명,차월,사전센싱대상자
코드,,,,,,,,,,,,,,,
0001870469,NaN,NaN,0,579.0,0.065948,2.0,True,False,False,5.0,15.0,5,NaN,NaN,0.0


In [27]:
grade_df = df.groupby(['grade']).agg(대상자수=('grade','size'),
                                    이탈자수=('y','sum'),
                                    평균점수=('score1','mean')
                                    ).reset_index()
grade_df = grade_df.sort_values(by='grade', ascending=False)
grade_df['이탈률'] = grade_df.이탈자수/grade_df.대상자수
grade_df

,grade,대상자수,이탈자수,평균점수,이탈률
7,8,697,0,687.350072,0.000000
6,7,675,0,647.008889,0.000000
5,6,713,2,617.957924,0.002805
4,5,704,2,584.187500,0.002841
3,4,705,4,538.648227,0.005674
2,3,705,7,468.055319,0.009929
1,2,696,44,363.076149,0.063218
0,1,705,123,300.251064,0.174468


In [9]:
df.score1.mean()

524.3777614138439

# 분류별 이탈률

In [10]:
grade_df = df.groupby(['case']).agg(대상자수=('case','size'),
                                    이탈자수=('y','sum'),
                                    평균점수=('score1','mean')
                                    ).reset_index()
# grade_df = grade_df.sort_values(by='case')
grade_df['이탈률'] = grade_df.이탈자수/grade_df.대상자수
grade_df

,case,대상자수,이탈자수,평균점수,이탈률
0,1.0,1101,3,667.842870,0.002725
1,2.0,1424,10,615.125000,0.007022
2,3.0,90,0,611.177778,0.000000
3,4.0,12,0,549.833333,0.000000
4,5.0,1146,21,521.788831,0.018325
5,6.0,9,0,502.444444,0.000000
6,7.0,66,5,488.484848,0.075758
7,8.0,1584,415,341.446338,0.261995


# 사전센싱 선정 대상자

In [11]:
tot = raw[(raw['2308차월']>7)&(raw['2308차월']<38)&(raw.해촉사유!='내용증명')&(raw.해촉사유!='사망')]
tot = tot[['코드','해촉일자','해촉사유']].set_index('코드')
tot['y'] = list(map(lambda x: 1 if x=='일반' else 0, tot.해촉사유))
tot.head(1)

,해촉일자,해촉사유,y
코드,,,
0001870469,NaN,NaN,0


In [12]:
tot.해촉사유.value_counts()

일반    3561
Name: 해촉사유, dtype: int64

In [13]:
program_lst = pd.read_csv('./사전센싱대상자/대상자2308.csv', converters={'FC코드':str}, delimiter='\t')
program_lst['사전센싱대상자']=1
program_lst = program_lst.set_index('FC코드')

score = pd.read_excel('../result2차/스코어_2308.xlsx', converters={'코드':str})
score = score[(score['2308차월']>=8)&(score['2308차월']<=37)]

score['grade'] = pd.cut(score['score1'], bins=bins, labels=labels, include_lowest=True)
score = score.set_index('코드')

df = tot.join(score).join(program_lst)
df = df[~df.score1.isna()]
df['사전센싱대상자'] = df.사전센싱대상자.fillna(0)
df.shape

(5432, 15)

In [20]:
grade_df = df.groupby(['grade']).agg(전체대상자=('그룹','size'),
                                    사전센싱대상자=('사전센싱대상자', 'sum'),
                                    이탈자수=('y','sum'),
                                    이탈률=('y','mean'),
                                      ).reset_index()
grade_df.sort_values(by='grade',ascending=False)

,grade,전체대상자,사전센싱대상자,이탈자수,이탈률
3,4,1222,118.0,0,0.000000
2,3,1601,211.0,5,0.003123
1,2,1489,165.0,15,0.010074
0,1,1288,24.0,162,0.125776


In [21]:
df2 = df[df.사전센싱대상자==1]
grade_df = df2.groupby(['grade']).agg(전체대상자=('그룹','size'),
                                    사전센싱대상자=('사전센싱대상자', 'sum'),
                                    이탈자수=('y','sum'),
                                    이탈률=('y','mean'),
#                              내용증명=('내용증명','sum')
                                      ).reset_index()
grade_df.sort_values(by='grade',ascending=False)

,grade,전체대상자,사전센싱대상자,이탈자수,이탈률
3,4,118,118.0,0,0.000000
2,3,211,211.0,1,0.004739
1,2,165,165.0,0,0.000000
0,1,24,24.0,1,0.041667


In [16]:
df.groupby(['case']).agg(전체대상자=('그룹','size'),
                        사전센싱대상자=('사전센싱대상자', 'sum'),
                        이탈자수=('y','sum'),
                        이탈률=('y','mean'),
                                      ).reset_index()

,case,전체대상자,사전센싱대상자,이탈자수,이탈률
0,1.0,1101,97.0,3,0.002725
1,2.0,1424,287.0,10,0.007022
2,3.0,90,6.0,0,0.000000
3,4.0,12,4.0,0,0.000000
4,5.0,1146,73.0,21,0.018325
5,6.0,9,2.0,0,0.000000
6,7.0,66,18.0,5,0.075758
7,8.0,1584,43.0,415,0.261995


# 사전센싱 대상자 2309

In [17]:
tot = raw[(raw['2309차월']>7)&(raw['2309차월']<38)&(raw.해촉사유!='내용증명')&(raw.해촉사유!='사망')]
tot = tot[['코드','해촉일자','해촉사유']].set_index('코드')
tot['y'] = list(map(lambda x: 1 if x=='일반' else 0, tot.해촉사유))


program_lst = pd.read_csv('./사전센싱대상자/대상자2309.csv', converters={'FC코드':str}, delimiter='\t')
program_lst['사전센싱대상자']=1
program_lst = program_lst.set_index('FC코드')

score = pd.read_excel('../result2차/스코어_2309.xlsx', converters={'코드':str})
score = score[(score['2309차월']>=8)&(score['2309차월']<=37)]

score['grade'] = pd.cut(score['score1'], bins=bins, labels=labels, include_lowest=True)
score = score.set_index('코드')

df = tot.join(score).join(program_lst)
df = df[~df.score1.isna()]
df['사전센싱대상자'] = df.사전센싱대상자.fillna(0)
df.shape

(5600, 15)

In [18]:
grade_df = df.groupby(['grade']).agg(전체대상자=('그룹','size'),
                        사전센싱대상자=('사전센싱대상자', 'sum'),
                        이탈자수=('y','sum'),
                        이탈률=('y','mean'),
                                      ).reset_index()
grade_df.sort_values(by='grade',ascending=False)

,grade,전체대상자,사전센싱대상자,이탈자수,이탈률
3,4,1222,118.0,0,0.000000
2,3,1601,211.0,5,0.003123
1,2,1489,165.0,15,0.010074
0,1,1288,24.0,162,0.125776


In [19]:
df.groupby(['case']).agg(전체대상자=('그룹','size'),
                        사전센싱대상자=('사전센싱대상자', 'sum'),
                        이탈자수=('y','sum'),
                        이탈률=('y','mean'),
                                      ).reset_index()

,case,전체대상자,사전센싱대상자,이탈자수,이탈률
0,1.0,1029,71.0,2,0.001944
1,2.0,1622,272.0,5,0.003083
2,3.0,60,4.0,0,0.000000
3,4.0,26,1.0,0,0.000000
4,5.0,1194,85.0,6,0.005025
5,6.0,6,2.0,0,0.000000
6,7.0,117,30.0,0,0.000000
7,8.0,1546,53.0,169,0.109314
